# Notebook to show Jpeg photos on a Map

Need to check the image properties each have latitudes and longitudes (I haven't added any error checking logic to handle images with missing location meta data).

In [ ]:
from base64 import b64encode
from io import BytesIO
from pathlib import Path

from ipyleaflet import Map, Marker, Popup
from ipywidgets import HTML, Layout
from PIL import Image
from PIL.ExifTags import GPSTAGS, TAGS

In [ ]:
def get_exif_data(image):
    """Returns a dictionary from the exif data of an PIL Image item. Also converts the GPS Tags"""
    exif_data = {}
    info = image._getexif()
    if info:
        for tag, value in info.items():
            decoded = TAGS.get(tag, tag)
            if decoded == "GPSInfo":
                gps_data = {}
                for gps_tag in value:
                    sub_decoded = GPSTAGS.get(gps_tag, gps_tag)
                    gps_data[sub_decoded] = value[gps_tag]

                exif_data[decoded] = gps_data
            else:
                exif_data[decoded] = value

    return exif_data
	
def _convert_to_degrees(value):
    d = value[0]
    m = value[1]
    s = value[2]
    return d + (m / 60.0) + (s / 3600.0)

def get_lat_lon(exif_data):
    """Returns the latitude and longitude, if available, from the provided exif_data (obtained through get_exif_data above)"""
    lat = None
    lon = None

    if "GPSInfo" in exif_data:		
        gps_info = exif_data["GPSInfo"]

        gps_latitude = gps_info.get("GPSLatitude")
        gps_latitude_ref = gps_info.get('GPSLatitudeRef')
        gps_longitude = gps_info.get('GPSLongitude')
        gps_longitude_ref = gps_info.get('GPSLongitudeRef')

        if gps_latitude and gps_latitude_ref and gps_longitude and gps_longitude_ref:
            lat = _convert_to_degrees(gps_latitude)
            if gps_latitude_ref != "N":                     
                lat *= -1

            lon = _convert_to_degrees(gps_longitude)
            if gps_longitude_ref != "E":
                lon *= -1

    return lat, lon

In [ ]:
image_file_iterator = Path(".").glob("*.jpg")
pil_images = [Image.open(image_file) for image_file in image_file_iterator]
exif_data_list = [get_exif_data(image) for image in pil_images]
lat_lngs = [get_lat_lon(exif) for exif in exif_data_list]

In [ ]:
min_lat = min(lat_lngs, key=lambda coord: coord[0])[0]
max_lat = max(lat_lngs, key=lambda coord: coord[0])[0]
min_lng = min(lat_lngs, key=lambda coord: coord[1])[1]
max_lng = max(lat_lngs, key=lambda coord: coord[1])[1]
lat_delta = max_lat-min_lat
lng_delta = max_lng-min_lng

In [ ]:
m = Map(layout=Layout(width='1024', height='768px'))
m.fit_bounds([[min_lat, min_lng],[max_lat, max_lng]])

for idx, image in enumerate(pil_images):
    f = BytesIO()
    image.save(f, "jpeg")
    data = b64encode(f.getvalue())
    data = data.decode('ascii')
    url = 'data:image/{};base64,'.format("jpeg") + data
    
    marker = Marker(location=lat_lngs[idx])
    m.add_layer(marker)
    message = HTML()
    message.value = f"""<img src={url} style="max-width:400px">""" + """<style>.leaflet-popup-content { 
     width:auto !important; 
}</style>"""
    
    # Popup has a max_width parameter but it doesn't seem to change the underlying CSS property
    # which is why width:auto !important; is set directly in the HTML style above
    popup = Popup(
        child=message,
        location=lat_lngs[idx]
    )

    marker.popup = popup

m